# Libraries

In [ ]:
import numpy as np
from matplotlib import pyplot
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
from sklearn.metrics import accuracy_score,  precision_score, confusion_matrix
from sklearn.metrics import classification_report

%matplotlib inline
plt.style.use('ggplot')

In [ ]:
from arima import *
from custom_loss import *
from state_of_the_art import *

In [ ]:
custom_loss = custom_loss()
arima, real_price = arima_model()
state_of_the_art = state_of_the_art()

# Our model

In [ ]:
result = pd.read_csv('result.csv')
model = result.prediction

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(result.label, label='BTC price')
plt.plot(result.prediction, label='Predicted')

plt.legend(['BTC price', 'Predicted'])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Price', fontsize=18, fontweight='bold')
plt.title("Project's sentiment model Bitcoin price prediction", fontsize=20, fontweight='bold')
fig.savefig('project_model.eps', dpi=600, bbox_inches='tight')
plt.show()

## Buy Sell Graphs

In [ ]:
result = pd.read_csv('result.csv')
df = pd.read_csv('price_sentiment_btc.csv')

In [ ]:
inter = pd.DataFrame(result.label, columns=['label'])
inter['prediction'] = result.prediction
result = inter.copy()

In [ ]:
from datetime import datetime
time_list = []
for i in df['Time']:
  dates = datetime.strptime(i, "%Y-%m-%d %H:%M:%S")
  date = f"{dates.date()}"
  time_list.append(date)
df['Time'] = time_list
df = df[-2320:].reset_index(drop=True)
result['Time'] = df['Time']

In [ ]:
def get_decisions(signs):
    decision = {'start': signs.index[0], 'end': None, 'val': signs[0]}
    decisions = [decision]
    for t in signs.index:
        if decisions[-1]['end'] != None:
            decision = {'start': next_start, 'end': None, 'val': signs[t]}
            decisions.append(decision)

        val = signs[t]
        if decisions[-1]['val'] == val:
            continue
        else:
            decisions[-1]['end'] = t
            next_start = t

    decisions[-1]['end'] = signs.index[-1]

    return decisions

In [ ]:
result = result.reset_index(drop=True)

In [ ]:
predicted_movement = []
actual_movement = []
for i in range(0, len(result.prediction)-1):
    if result.prediction[i+1] > result.prediction[i]:
        predicted_movement.append(1)
    else:
        predicted_movement.append(-1)
    if result.label[i+1] > result.label[i]:
        actual_movement.append(1)
    else:
        actual_movement.append(-1)

In [ ]:
price_df = pd.DataFrame(result.label)
price_df['Time'] = result['Time']
price_df = price_df.set_index('Time')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,7))
fig.tight_layout()
fig.subplots_adjust(hspace=0.4)

# Moving Average Crossover
price = price_df.label

move_signs = pd.Series(np.sign(predicted_movement))
signs = move_signs.copy()
idx = move_signs.index[0]

# crossover plot 
price[idx:].plot(ax=ax, label='BTC Close price')
decisions = get_decisions(signs)
for dec in decisions:
    ax.axvspan(dec['start'], dec['end'], alpha=0.4, color = 'skyblue' if dec['val'] == 1 else 'salmon')
ax.legend()
ax.set_title("Bitcoin trading strategy", fontsize=20, fontweight='bold')
ax.set_ylabel('Bitcoin Price', fontsize=18, fontweight='bold')
ax.set_xlabel('Date', fontsize=18, fontweight='bold')
fig.savefig('15_BTC_trading_strategy_plot.eps', dpi=600, bbox_inches='tight')

In [ ]:
hourly_labels = []
hourly_predictions = []
for i in range(0, len(result.label),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(result.prediction[i])


predicted_movement = []
actual_movement = []
for i in range(0, len(hourly_predictions)-1):
    if hourly_predictions[i+1] > hourly_predictions[i]:
        predicted_movement.append(1)
    else:
        predicted_movement.append(-1)
    if hourly_labels[i+1] > hourly_labels[i]:
        actual_movement.append(1)
    else:
        actual_movement.append(-1)

In [ ]:
price_df = pd.DataFrame(hourly_labels, columns=['label'])
price_df['Time'] = result['Time']
price_df = price_df.set_index('Time')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,7))
fig.tight_layout()
fig.subplots_adjust(hspace=0.4)

# Moving Average Crossover
price = price_df.label

move_signs = pd.Series(np.sign(predicted_movement))
signs = move_signs.copy()
idx = move_signs.index[0]

# crossover plot 
price[idx:].plot(ax=ax, label='BTC Close price')
decisions = get_decisions(signs)
for dec in decisions:
    ax.axvspan(dec['start'], dec['end'], alpha=0.4, color = 'skyblue' if dec['val'] == 1 else 'salmon')
ax.legend()
ax.set_title("Bitcoin trading strategy", fontsize=20, fontweight='bold')
ax.set_ylabel('Bitcoin Price', fontsize=18, fontweight='bold')
ax.set_xlabel('Date', fontsize=18, fontweight='bold')
fig.savefig('hourly_BTC_trading_strategy_plot.eps', dpi=600, bbox_inches='tight')

# Compare

In [ ]:
def buy_sell_trades(actual, predicted, threshold):
    pred_df = pd.DataFrame()
    pred_df['Predictions'] = predicted
    threshold /= 100


    y_pct_change = pred_df.pct_change()

    money = 10000
    number_of_stocks = (10000 / actual[0])
    left = (((actual[len(actual)-1]-actual[0])/actual[0])+1)*10000
    trades = 0
    number_of_stocks = 0
    portfolio = []
    bh = []

    buying_percentage_threshold = threshold 
    selling_percentage_threshold = threshold

    for i in range(len(predicted) - 1):    
        if y_pct_change['Predictions'][i + 1] > buying_percentage_threshold:
            for j in np.arange(1, 0, -0.001):
                #Buying of stock
                if (money >= j * actual[i]):
                    trades += 1
                    money -= j * actual[i]
                    number_of_stocks += (j*(1-0.001))
                    break
        elif  y_pct_change['Predictions'][i + 1] < -selling_percentage_threshold:
            for j in np.arange(1, 0, -0.001):
                #Selling of stock
                if (number_of_stocks >= j):
                    money += (j*(1-0.001)) * actual[i]
                    number_of_stocks -= j
                    break
        portfolio.append((number_of_stocks * actual[i])+money)
        bh.append(actual[i])

    money += number_of_stocks * actual[len(actual) - 1]

    print('Number of trades: ', trades)

    return portfolio, trades


In [ ]:
def no_fees_buy_sell_trades(actual, predicted, threshold):
    pred_df = pd.DataFrame()
    pred_df['Predictions'] = predicted
    threshold /= 100


    y_pct_change = pred_df.pct_change()

    money = 10000
    number_of_stocks = (10000 / actual[0])
    left = (((actual[len(actual)-1]-actual[0])/actual[0])+1)*10000
    trades = 0
    number_of_stocks = 0
    portfolio = []
    bh = []

    buying_percentage_threshold = threshold 
    selling_percentage_threshold = threshold

    for i in range(len(predicted) - 1):    
        if y_pct_change['Predictions'][i + 1] > buying_percentage_threshold:
            for j in np.arange(1, 0, -0.001):
                #Buying of stock
                if (money >= j * actual[i]):
                    trades += 1
                    money -= j * actual[i]
                    number_of_stocks += (j*(1))
                    break
        elif  y_pct_change['Predictions'][i + 1] < -selling_percentage_threshold:
            for j in np.arange(1, 0, -0.001):
                #Selling of stock
                if (number_of_stocks >= j):
                    money += (j*(1)) * actual[i]
                    number_of_stocks -= j
                    break
        portfolio.append((number_of_stocks * actual[i])+money)
        bh.append(actual[i])

    money += number_of_stocks * actual[len(actual) - 1]

    print('Number of trades: ', trades)

    return portfolio, trades


In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(real_price, label='BTC price')
plt.plot(custom_loss, label='Custom loss')
plt.plot(state_of_the_art, label='State of the art')
plt.plot(model-2000, label='Project sentiment model')

plt.legend(['BTC price', 'Custom loss', 'State of the art', 'Project sentiment model'])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Price', fontsize=18, fontweight='bold')
plt.title('Actual price and predicted price for test data', fontsize=20, fontweight='bold')
fig.savefig('compare_models.eps', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
strat_arima, arima_trades = no_fees_buy_sell_trades(real_price, arima, 0)
strat_custom_loss, custom_trades = no_fees_buy_sell_trades(real_price, custom_loss, 0)
strat_state_of_the_art, state_trades = no_fees_buy_sell_trades(real_price, state_of_the_art, 0)
strat_model, model_trades = no_fees_buy_sell_trades(real_price ,model, 0)

In [ ]:
hold = [10000]
r = pd.DataFrame(real_price)
price = r['True'].pct_change()
for i in range(1,len(price)-1):
  hold.append(hold[i-1] * (1+price[i]))

In [ ]:
hourly_labels = []
hourly_predictions = []
for i in range(0, len(result.label),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(result.prediction[i])
strat_hourly = buy_sell_trades(hourly_labels, hourly_predictions, 0)

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(hold, label='Buy and hold')
plt.plot(strat_custom_loss, label='Custom loss model')
plt.plot(strat_state_of_the_art, label='State of the art')
plt.plot(strat_model, label="Project's sentiment model")

plt.legend(['Buy and hold', 'Custom loss model', 'State of the art', "Project's sentiment model"])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Account value', fontsize=18, fontweight='bold')
plt.title('Comparison of models performances (no fees)', fontsize=20, fontweight='bold')
fig.savefig('compare_no_fees.eps', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
strat_arima, arima_trades = no_fees_buy_sell_trades(real_price, arima, 0.2)
strat_custom_loss, arima_trades = no_fees_buy_sell_trades(real_price, custom_loss, 0.2)
strat_state_of_the_art, arima_trades = no_fees_buy_sell_trades(real_price, state_of_the_art, 0.2)
strat_model, arima_trades = no_fees_buy_sell_trades(real_price ,model, 0.2)

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(hold, label='Buy and hold')
plt.plot(strat_custom_loss, label='Custom loss model')
plt.plot(strat_state_of_the_art, label='State of the art')
plt.plot(strat_model, label="Project's sentiment model")

plt.legend(['Buy and hold', 'Custom loss model', 'State of the art', "Project's sentiment model"])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Account value', fontsize=18, fontweight='bold')
plt.title('Comparison of models performances (no fees with 0.2% threshold)', fontsize=20, fontweight='bold')
fig.savefig('compare_no_fees_threshold.eps', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
strat_arima, arima_trades = buy_sell_trades(real_price, arima, 0)
strat_custom_loss, arima_trades = buy_sell_trades(real_price, custom_loss, 0)
strat_state_of_the_art, arima_trades = buy_sell_trades(real_price, state_of_the_art, 0)
strat_model, arima_trades = buy_sell_trades(real_price ,model, 0)

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(hold, label='Buy and hold')
plt.plot(strat_custom_loss, label='Custom loss model')
plt.plot(strat_state_of_the_art, label='State of the art')
plt.plot(strat_model, label="Project's sentiment model")

plt.legend(['Buy and hold', 'Custom loss model', 'State of the art', "Project's sentiment model"])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Account value', fontsize=18, fontweight='bold')
plt.title('Comparison of models performances (with fees)', fontsize=20, fontweight='bold')
fig.savefig('compare_fees.eps', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
strat_arima, arima_trades = buy_sell_trades(real_price, arima, 0.2)
strat_custom_loss, arima_trades = buy_sell_trades(real_price, custom_loss, 0.2)
strat_state_of_the_art, arima_trades = buy_sell_trades(real_price, state_of_the_art, 0.2)
strat_model, arima_trades = buy_sell_trades(real_price ,model, 0.2)

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(hold, label='Buy and hold')
plt.plot(strat_custom_loss, label='Custom loss model')
plt.plot(strat_state_of_the_art, label='State of the art')
plt.plot(strat_model, label="Project's sentiment model")

plt.legend(['Buy and hold', 'Custom loss model', 'State of the art', "Project's sentiment model"])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Account value', fontsize=18, fontweight='bold')
plt.title('Comparison of models performances (with fees and 0.2% threshold)', fontsize=20, fontweight='bold')
fig.savefig('compare_fees_threshold.eps', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# hourly 
# own model
hourly_labels = []
hourly_predictions = []
for i in range(0, len(result.label),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(result.prediction[i])
hourly_model, arima_trades = buy_sell_trades(hourly_labels, hourly_predictions, 0)

#custom 
hourly_labels = []
hourly_predictions = []
for i in range(0, len(custom_loss),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(custom_loss[i])
hourly_custom, arima_trades = buy_sell_trades(hourly_labels, hourly_predictions, 0)

# state
hourly_labels = []
hourly_predictions = []
for i in range(0, len(state_of_the_art),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(state_of_the_art[i])
hourly_state, arima_trades = buy_sell_trades(hourly_labels, hourly_predictions, 0)

In [ ]:
# hold
hourly_hold = [10000]
r = pd.DataFrame(real_price)
price = r['True'].pct_change()
j=0
for i in range(1,len(price)-5, 4):
  hourly_hold.append(10000)
  j +=1
  hourly_hold[j] = (hourly_hold[j-1] * (1+price[i]))

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(hourly_hold, label='Buy and hold')
plt.plot(hourly_custom, label='Custom loss model')
plt.plot(hourly_state, label='State of the art')
plt.plot(hourly_model, label="Project's sentiment model")

plt.legend(['Buy and hold', 'Custom loss model', 'State of the art', "Project's sentiment model"])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Account value', fontsize=18, fontweight='bold')
plt.title('Comparison of hourly models performances (with fees)', fontsize=20, fontweight='bold')
fig.savefig('compare_hourly_fees.eps', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
predicted = hourly_state
actual = hourly_hold

In [ ]:
predicted_movement = []
actual_movement = []
for i in range(0, len(predicted)-1):
    if predicted[i+1] > predicted[i]:
        predicted_movement.append(1)
    else:
        predicted_movement.append(0)
    if actual[i+1] > actual[i]:
        actual_movement.append(1)
    else:

In [ ]:
report_test = classification_report(actual_movement, predicted_movement)
print(report_test)

In [ ]:
# hourly 
# own model
hourly_labels = []
hourly_predictions = []
for i in range(0, len(result.label),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(result.prediction[i])
hourly_model, arima_trades = buy_sell_trades(hourly_labels, hourly_predictions, 0.2)

#custom 
hourly_labels = []
hourly_predictions = []
for i in range(0, len(custom_loss),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(custom_loss[i])
hourly_custom, arima_trades = buy_sell_trades(hourly_labels, hourly_predictions, 0.2)

# state
hourly_labels = []
hourly_predictions = []
for i in range(0, len(state_of_the_art),4):
    hourly_labels.append(result.label[i])
    hourly_predictions.append(state_of_the_art[i])
hourly_state, arima_trades = buy_sell_trades(hourly_labels, hourly_predictions, 0.2)

In [ ]:
fig = plt.figure(figsize = (16, 7))
plt.plot(hourly_hold, label='Buy and hold')
plt.plot(hourly_custom, label='Custom loss model')
plt.plot(hourly_state, label='State of the art')
plt.plot(hourly_model, label="Project's sentiment model")

plt.legend(['Buy and hold', 'Custom loss model', 'State of the art', "Project's sentiment model"])
plt.xlabel('Time', fontsize=18, fontweight='bold')
plt.ylabel('Account value', fontsize=18, fontweight='bold')
plt.title('Comparison of hourly models performances (with fees with 0.2% threshold)', fontsize=20, fontweight='bold')
fig.savefig('compare_hourly_fees_threshold.eps', dpi=600, bbox_inches='tight')
plt.show()